# Работа с данными сервиса "Яндекс Книги" в ClickHouse

- Автор: Логинов Павел Александрович
- Дата: 28.10.2025

## Краткое описание проекта

Нас взяли на стажировку в сервис Яндекс Книги. В бэклоге у продакт-менеджера накопились ad hoc задачи по аналитике. Они посвящены основным метрикам и паттернам использования сервиса.

Основной инструмент решения задач — **ClickHouse**. Понадобится проанализировать в основном мобильные платформы `'Букмейт iOS'` и `'Букмейт Android'`, а в некоторых случаях и `'Букмейт Web'`, если это явно указано.

Для анализа необходимо использовать две таблицы — `source_db.audition` и `source_db.content`. 

## Описание данных

1) Таблица `source_db.audition` - данные о прослушиваниях/прочтениях:

- `audition_id` (тип данных: **Int32**) - идентификатор прослушивания или прочтения
- `puid` (тип данных: **String**) - идентификатор пользователя
- `usage_platform_ru` (тип данных: **String**) - платформа, в которой произошло прослушивание или прочтение
- `msk_business_dt_str` (тип данных: **Date**) - дата прослушивания или прочтения
- `app_version` (тип данных: **Nullable(String)**) - версия платформы
- `adult_content_flg` (тип данных: **Bool**) - контент для взрослых или нет
- `hours` (тип данных: **Float32**) - длительность прослушивания или прочтения
- `hours_sessions_long` (тип данных: **Float32**) - длительность всей сессии
- `kids_content_flg` (тип данных: **Bool**) - контент для детей или нет
- `main_content_id` (тип данных: **String**) - идентификатор контента (общий столбец с таблицей `source_db.content`)
- `usage_geo_id_name` (тип данных: **String**) - город прослушивания или прочтения
- `usage_country_name` (тип данных: **String**) - страна прослушивания или прочтения

2) Таблица `source_db.content` - данные о контенте:

- `main_content_id` (тип данных: **String**) - идентификатор контента (общий столбец с таблицей `source_db.audition`)
- `main_content_type` (тип данных: **String**) - тип контента
- `main_content_name` (тип данных: **String**) - название контента
- `main_content_duration_hours` (тип данных: **Float32**) - время прослушивания или прочтения контента
- `main_author_name` (тип данных: **String**) - имя автора контента
- `published_topic_title_list` (тип данных: **Array(String)**) - список тегов в виде массива данных

### Задание №1

Для начала продакт-менеджер хочет понять, где сервис пользуется наибольшей популярностью. Выведем топ-20 городов и регионов России по суммарному количеству прочитанных и прослушанных часов любого контента с мобильных устройств. Для каждой из платформ — iOS и Android — добавим отдельный столбец с длительностью. Результат будет выглядеть так: город, общая длительность прочитанного и прослушанного контента, длительность на iOS, длительность на Android. Значения округлим до целых чисел для лучшей читаемости. Из выдачи также исключим федеральные округа — оставим только города и области.

In [ ]:
SELECT 
    usage_geo_id_name AS "Город",
    round(sum(hours)) AS "Общая длительность",
    round(sumIf(hours, usage_platform_ru = 'Букмейт iOS')) AS "Длительность контента на iOS",
    round(sumIf(hours, usage_platform_ru = 'Букмейт Android')) AS "Длительность контента на Android"
FROM source_db.audition 
WHERE usage_country_name = 'Россия' AND 
      usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android') AND
      usage_geo_id_name NOT LIKE '%округ%'
GROUP BY "Город"
ORDER BY "Общая длительность" DESC
LIMIT 20;

**Вывод:**

Наибольшей популярностю сервис пользуется в `Москве` (**26629** часов прослушивания/прочитывания), `Санкт-Петербурге` (**12538** часов прослушивания/прочитывания), а также в `Москве и Московской области` (**8248** часов прослушивания/прочитывания). При этом длительность прослушанного и прочитанного контента для пользователей Android выше, чем для пользоватлей iOS примерно в два раза для каждого из регионов России

### Задание №2

С активными регионами определились, а какой контент самый популярный? Получим топ-5 книг по суммарному количеству прочитанных и прослушанных часов на мобильных платформах. Также вычислим среднее время чтения и прослушивания в зависимости от типа книги: текст или аудио. Результат будет выглядеть так: название книги, её автор, суммарное время чтения и прослушивания, среднее время чтения текстовой книги, среднее время прослушивания аудиокниги.

В список включим только те книги, которые используются в обоих форматах. Числовые значения округлим до двух знаков после точки.

In [ ]:
WITH both_formats_books AS (
    SELECT main_content_name, main_author_name
    FROM source_db.content 
    WHERE main_content_type IN ('Book', 'Comicbook', 'Audiobook')
    GROUP BY main_content_name, main_author_name
    HAVING count(distinct main_content_type) >= 2
)
SELECT 
    c.main_content_name AS "Название книги",
    c.main_author_name AS "Автор",
    round(sum(a.hours), 2) AS "Суммарное время прослушивания/прочтения",
    round(avgIf(a.hours, c.main_content_type IN ('Book', 'Comicbook')), 2) AS "Среднее время прочтения",
    round(avgIf(a.hours, c.main_content_type = 'Audiobook'), 2) AS "Среднее время прослушивания"
FROM source_db.audition a
INNER JOIN source_db.content c ON a.main_content_id = c.main_content_id
WHERE a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android') AND 
     (c.main_content_name, c.main_author_name) IN (SELECT main_content_name, main_author_name FROM both_formats_books)
GROUP BY "Название книги", "Автор"
ORDER BY "Суммарное время прослушивания/прочтения" DESC
LIMIT 5;

**Вывод:**

Топ-5 книг по суммарному количеству прочитанных и прослушанных часов на мобильных платформах составляет:

- `Илон Маск` (суммарное время прослушивания: **1012** часов)
- `Железное пламя` (суммарное время прослушивания: **781** час)
- `Убийства и кексики. Детективное агентство «Благотворительный магазин»` (суммарное время прослушивания: **541** час)
- `Четвертое крыло` (суммарное время прослушивания: **501** час)
- `Земля лишних. Трилогия` (суммарное время прослушивания: **481** час)

При этом показатели среднего времени прочтения не имеют прямой зависимости от суммарного времени прослушивания/прочтения, а показатели среднего времени прослушивания - имеют обратную зависимость от суммарного времени прослушивания/прочтения: чем меньше суммарное время прослушивания/прочтения, тем меньше среднее время прослушивания

### Задание №3

Составим топ-10 авторов по суммарной длительности чтения их книг на всех платформах, включая веб. Для каждого автора добавим количество уникальных текстовых книг (тип контента `'Book'`) и выведем среднюю длительность прослушивания их аудиокниг только на мобильных устройствах. Исключим авторов, у которых нет аудиокниг.

In [ ]:
SELECT 
    c.main_author_name AS "Автор",
    round(sum(a.hours)) AS "Общая длительность",
    count(distinct if(c.main_content_type = 'Book', c.main_content_id, NULL)) AS "Количество текстовых книг",
    round(avgIf(a.hours, c.main_content_type = 'Audiobook' AND 
                         a.usage_platform_ru LIKE '%iOS%' or a.usage_platform_ru LIKE '%Android%'), 2) AS "Средняя длительность прослушивания (только мобильные устройства)"
FROM source_db.audition a
INNER JOIN source_db.content c ON a.main_content_id = c.main_content_id
WHERE c.main_author_name IN (
    SELECT DISTINCT main_author_name 
    FROM source_db.content 
    WHERE main_content_type = 'Audiobook'
) AND a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android', 'Букмейт Web')
GROUP BY "Автор"
ORDER BY "Общая длительность" DESC
LIMIT 10;

**Вывод:**

Топ-10 авторов по суммарной длительности чтения их книг на всех платформах, включая веб, составляет:

- `Сергей Лукьяненко` (общая длительность прочтения книг: **3202** часа)
- `Александра Лисина` (общая длительность прочтения книг: **2243** часа)
- `Анджей Сапковский` (общая длительность прочтения книг: **2156** часов)
- `Дарья Донцова` (общая длительность прочтения книг: **2096** часов)
- `Борис Акунин` (общая длительность прочтения книг: **2041** час)
- `Фёдор Достоевский` (общая длительность прочтения книг: **1744** часа)
- `Артём Каменистый` (общая длительность прочтения книг: **1708** часов)
- `Макс Фрай` (общая длительность прочтения книг: **1664** часов)
- `Виктор Пелевин` (общая длительность прочтения книг: **1630** часов)
- `Андрей Круз` (общая длительность прочтения книг: **1519** часов)

При этом показатели количества текстовых книг и среднего прослушивания аудиокниг авторов только на мобильных устройствах не имеют прямой зависимости от общей длительности чтения книг авторах на всех платформах, включая веб

### Задание №4

У продакт-менеджера есть предположение, что среди Android-пользователей аудиокниги почти так же популярны, как тексты. А среди iOS-пользователей читателей книг вдвое больше, чем слушателей, если считать по суммарной длительности сессии.

Проверим предположение менеджера. Для начала выделим три сегмента пользователей:

- «Слушатель» — тот, кто преимущественно пользуется аудиокнигами. Прослушивание книг составляет 70% и выше от суммарной длительности сессий.
- «Читатель» — преимущественно пользуется текстовыми книгами. Чтение книг — от 70%.
- «Оба» — остальные пользователи сервиса.

Исключим пользователей, у которых нет сессий ни с книгами, ни с аудиокнигами, и посчитайте количество пользователей в каждом из сегментов.

На основе полученных данных проверим предположение менеджера о том, что среди пользователей Android примерно одинаковое количество читателей и слушателей, а на устройствах iOS читателей книг вдвое больше, чем слушателей. Чтобы определить основную платформу пользователя, учтём время её использования. Например, если пользователь посещал сервис с двух устройств: два часа на iOS и пять часов на Android, то основной платформой такого пользователя будет Android.

In [ ]:
WITH user_stats AS (
    SELECT 
        puid,
        argMax(usage_platform_ru, hours) AS main_platform,
        sumIf(hours, c.main_content_type IN ('Book', 'Comicbook')) AS reading_hours,
        sumIf(hours, c.main_content_type = 'Audiobook') AS listening_hours
    FROM source_db.audition a
    INNER JOIN source_db.content c ON a.main_content_id = c.main_content_id
    WHERE c.main_content_type IN ('Book', 'Comicbook', 'Audiobook') AND 
          a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
    GROUP BY puid
    HAVING (reading_hours + listening_hours) > 0
)
SELECT 
    main_platform AS "Платформа",
    multiIf(
    listening_hours / (reading_hours + listening_hours) >= 0.7, 'Слушатель',
    reading_hours / (reading_hours + listening_hours) >= 0.7, 'Читатель',
    'Оба') AS "Сегмент",
    count(*) AS "Количество пользователей"
FROM user_stats
GROUP BY "Платформа", "Сегмент"
ORDER BY "Платформа", "Количество пользователей" DESC;

**Вывод:**

Среди `Android-пользователей` аудиокниги менее популярны, чем текстовые книги (**2500** - читают, **2000** - слушают). Среди `iOS-пользователей` читателей больше, чем слушателей, но не вдвое, а лишь на треть (**2000** - читают, **1400** - слушают). Таким образом, мы делаем вывод о том, что предположение менеджера - не подтвердилось, так как `Android-пользователи` значительно больше читают (нежели слушают), а для `iOS-пользователей` разница между читателями и слушателями оказалась меньше ожидаемой

### Задание №5

Изучим, существует ли связь между форматом использования приложения (прослушивание или чтение) и днём недели. Падает ли использование аудиокниг в выходные на всех платформах, включая веб? Чтобы это узнать, для каждого типа контента посчитаем среднее время его использования в рабочие и выходные дни.

In [ ]:
SELECT DISTINCT
    c.main_content_type AS "Тип контента",
    avgIf(hours, toDayOfWeek(a.msk_business_dt_str) IN (6, 7)) AS "Среднее время прослушивания/прочтения (выходные дни)",
    avgIf(hours, toDayOfWeek(a.msk_business_dt_str) NOT IN (6, 7)) AS "Среднее время прослушивания/прочтения (рабочие дни)"
FROM source_db.audition a
INNER JOIN source_db.content c ON a.main_content_id = c.main_content_id
WHERE c.main_content_type IN ('Book', 'Comicbook', 'Audiobook') AND
      a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android', 'Букмейт Web')
GROUP BY "Тип контента"
ORDER BY "Тип контента";

**Вывод:**

- Для `аудиокниг` среднее время прослушивания выше в выходные дни, чем в рабочие - на **0.06** часа.
- Для `книг` и `комиксов` среднее время прочтения выше в выходные дни, чем в рабочие дни - на **0.7** и **0.3** часа соотвественно.

Соответственно, мы делаем вывод о том, что пользователи больше слушают и читают по выходным, чем в рабочие дни. А значит, что связь между форматом использования приложения (прослушивание или чтение) и днём недели - существует. Падение использования аудиокниг на всех платформах, включая веб - не наблюдается

### Задание №6

Продакт-менеджер хочет отслеживать обновления приложений на Android и iOS. У него есть предположение, что больший процент пользователей iOS используют последнюю версию приложения и в целом чаще его обновляют.

Для начала изучим, у какой части пользователей на текущий момент стоят последние версии приложения на каждой из платформ. Для этого посчитаем последнюю активную версию каждого пользователя и сравним её с последней версией у каждой платформы. Для каждой платформы выведем процент пользователей с последней версией приложения и округлим его до двух знаков после точки.

In [ ]:
WITH user_last_versions AS (
    SELECT 
        usage_platform_ru,
        puid,
        argMax(app_version, msk_business_dt_str) AS last_version
    FROM source_db.audition
    WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android') AND 
          app_version IS NOT NULL
    GROUP BY usage_platform_ru, puid
),
platform_max AS (
    SELECT 
        usage_platform_ru,
        max(last_version) AS latest_version
    FROM user_last_versions
    GROUP BY usage_platform_ru
)
SELECT 
    ulv.usage_platform_ru AS "Платформа",
    round(countIf(ulv.puid, ulv.last_version = pm.latest_version) * 100.0 / count(ulv.puid), 2) AS "Доля пользователей с последней версией"
FROM user_last_versions ulv
INNER JOIN platform_max pm ON ulv.usage_platform_ru = pm.usage_platform_ru
GROUP BY "Платформа";

**Вывод:**

Пользователи `Android` обновляют приложение `Букмейт` на **27%** чаще по сравнению с пользователями `iOS`

Таким образом, мы делаем вывод о том, что предположение менеджера о том, что больший процент пользователей `iOS` используют последнюю версию приложения и в целом чаще его обновляют - не подтверждается

### Задание №7

Теперь продакт-менеджер хочет понять, как часто пользователи обновляют приложение на каждой из платформ. Фактом обновления будем считать изменение версии у каждого пользователя. Представим, что любое изменение возможно только в сторону более новой версии.

Проверим предположение о том, что пользователи iOS чаще обновляют приложение. Посчитаем метрику, которая покажет среднюю частоту обновлений на пользователя. Округлим её до двух знаков после точки.

In [ ]:
WITH user_updates AS (
    SELECT 
        usage_platform_ru,
        puid,
        count(distinct app_version) - 1 AS update_count
    FROM source_db.audition
    WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android') AND 
          app_version IS NOT NULL
    GROUP BY usage_platform_ru, puid
    HAVING update_count >= 0
)
SELECT 
    usage_platform_ru AS "Платформа",
    round(avg(update_count), 2) AS "Средняя частота обновлений на пользователя"
FROM user_updates
GROUP BY "Платформа"
ORDER BY "Платформа";

**Вывод:**

Пользователи `Android` обновляют приложение `Букмейт` на **0.58** обновлений чаще по сравнению с пользователями `iOS`

Таким образом, мы делаем вывод о том, что предположение о том, что пользователи `iOS` чаще обновляют приложение - не подтвержается

### Задание №8

Новая задача — у коллег есть опасения, что не все книги на тему магии верно размечены с точки зрения категорий.  Считается, что у книги должно быть не больше 3–4 категорий с темами. Необходимо найти все книги на магическую тему, которые при этом не входят в художественную литературу, и проверить, правильно ли они размечены.

Начнём с подсчёта книг с тегом «Магия». Выведем количество таких книг в каталоге.

In [ ]:
SELECT count(distinct main_content_id) AS "Количество книг с тегом магия"
FROM source_db.content c
INNER JOIN source_db.audition a ON c.main_content_id = a.main_content_id
WHERE has(published_topic_title_list, 'Магия') AND
      NOT has (published_topic_title_list, 'Художественная литература') AND
      a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android');

**Вывод:**

Количество книг на магическую тему (с тегом `Магия`), которые не входят в художественную литературу, составляет **37** книг

### Задание №9

Найдём все книги со словом «магия» в названии, для которых не проставлен тег «Магия». При этом не будем учитывать книги с тегом «Художественная литература». Выведем количество таких книг в каталоге.

In [ ]:
SELECT count(distinct main_content_id) AS "Количество книг с непроставленным тегом 'Магия'"
FROM source_db.content c
INNER JOIN source_db.audition a ON c.main_content_id = a.main_content_id
WHERE lower(main_content_name) LIKE '%магия%' AND 
      NOT has(published_topic_title_list, 'Магия') AND 
      NOT has(published_topic_title_list, 'Художественная литература') AND
      a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android');

**Вывод:**

Количество книг, со словом `магия` в названии, для которых не проставлен тег `Магия` (без учёта книг с тегом `Художественная литература`), составляет **14** книг

Таким образом, мы делаем вывод о том, что опасения коллег о том, что не все книги на тему магии верно размечены с точки зрения категорий контента - подтвердилось, так как из **37** книг только **14** книг размечены верно с точки зрения категорий контента (ещё **23** книги остались без верной разметки с точки зрения категорий контента)

### Задание №10

Посчитаем среднее количество категорий у книг с тегом «Магия» и среднее количество категорий у книг в каталоге в целом. Округлим значения до двух знаков после точки. Напомним, что коллегам важно, чтобы у каждой книги было не больше 3–4 категорий. Получится ли не превысить рекомендованного количества?

In [ ]:
SELECT 
    ROUND(avgIf(length(published_topic_title_list), has(published_topic_title_list, 'Магия')), 2) AS "Среднее количество категорий у книг с тегом 'Магия'",
    ROUND(avg(length(published_topic_title_list)), 2) AS "Среднее количество категорий у книг в каталоге в целом"
FROM source_db.content c
INNER JOIN source_db.audition a ON c.main_content_id = a.main_content_id
WHERE a.usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android');

**Вывод:**

- Среднее количество категорий у книг с тегом `Магия` - **3.14**
- Среднее количество категорий у книг в каталоге в целом - **4.2**

Таким образом, мы делаем вывод о том, что у каждой книги примерно 4 категории в каталоге. Следовательно, рекомендованное количество - не превышено

### Задание №11

Продакт-менеджер выяснил, что в приложении одной из мобильных платформ могла возникнуть проблема — длина пользовательской сессии (столбец `hours_sessions_long`) записывается некорректно, и это происходит как минимум в одной из стран. Чтобы найти аномалию в данных, используем такую меру дисперсии как коэффициент вариации.

Исследуем коэффициент по странам и мобильным платформам. В какой стране и на какой платформе видна аномалия в данных? Ограничим выборку одной страной, в которой коэффициент вариации для одной из платформ будет наибольшим.

In [ ]:
SELECT 
    usage_country_name AS "Страна",
    usage_platform_ru AS "Платформа",
    ROUND(stddevPop(hours_sessions_long) / avg(hours_sessions_long), 4) AS "Коэффициент вариации"
FROM source_db.audition
WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android') AND 
      hours_sessions_long > 0
GROUP BY usage_country_name, usage_platform_ru
ORDER BY "Коэффициент вариации" DESC
LIMIT 1;

**Вывод:**

Рассчёт коэффициента вариации показал, что аномалия длины пользовательской сессии наблюдается в `Латвии`, на платформе `Букмейт Android` - коэффициент вариации составляет **7**, в то время как значения коэффициента вариации в других странах и платформах не превышает **2.4**. Это означает, что длина пользовательской сессии для `Android-пользователей Букмейт из Латвии` записывается некорректно